In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import pickle
import joblib

In [3]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [4]:
path = '/content/drive/MyDrive/DATASET/MachineLearningCVE/'

In [5]:
csv_files=["Monday-WorkingHours.pcap_ISCX",
        "Tuesday-WorkingHours.pcap_ISCX",
        "Wednesday-workingHours.pcap_ISCX",
        "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX",
        "Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX",
        "Friday-WorkingHours-Morning.pcap_ISCX",
        "Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX",
        "Friday-WorkingHours-Afternoon-DDos.pcap_ISCX",]

In [6]:
def dfAppendClean(csv_files):
  data = pd.DataFrame()
  for i in range(len(csv_files)):
    df = pd.read_csv(path + csv_files[i] + ".csv")
    data = data.append(df)
    data = data.fillna(0)
    data[' Label'] = data[' Label'].str.replace(r'[^\w\s]+', '')
    data = data.replace([np.inf, -np.inf], 0)
  return data

In [7]:
data = dfAppendClean(csv_files)
data.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.000000,0.000000,0,0,0.0,0.000000,3.000000e+06,5.000000e+05,4.00,0.000000,4,4,4,4.000,0.000000,4,4,0,0.000000,0.000000,0,0,0,0,0,0,40,0,5.000000e+05,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,0,9.000000,6.000000,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.000000,0.000000,0,0,0.0,0.000000,1.200000e+07,2.000000e+06,1.00,0.000000,1,1,1,1.000,0.000000,1,1,0,0.000000,0.000000,0,0,0,0,0,0,40,0,2.000000e+06,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,0,9.000000,6.000000,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.000000,0.000000,0,0,0.0,0.000000,1.200000e+07,2.000000e+06,1.00,0.000000,1,1,1,1.000,0.000000,1,1,0,0.000000,0.000000,0,0,0,0,0,0,40,0,2.000000e+06,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,0,9.000000,6.000000,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.000000,0.000000,0,0,0.0,0.000000,1.200000e+07,2.000000e+06,1.00,0.000000,1,1,1,1.000,0.000000,1,1,0,0.000000,0.000000,0,0,0,0,0,0,40,0,2.000000e+06,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,0,9.000000,6.000000,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.000000,0.000000,0,0,0.0,0.000000,4.000000e+06,6.666667e+05,3.00,0.000000,3,3,3,3.000,0.000000,3,3,0,0.000000,0.000000,0,0,0,0,0,0,40,0,6.666667e+05,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,0,9.000000,6.000000,0.0,40,0,0,0,0,0,0,2,12,0,0,245,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
5,49486,1,2,0,12,0,6,6,6.000000,0.000000,0,0,0.0,0.000000,1.200000e+07,2.000000e+06,1.00,0.000000,1,1,1,1.000,0.000000,1,1,0,0.000000,0.000000,0,0,0,0,0,0,40,0,2.000000e+06,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,0,9.000000,6.000000,0.0,40,0,0,0,0,0,0,2,12,0,0,245,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
6,49486,1,2,0,12,0,6,6,6.000000,0.000000,0,0,0.0,0.000000,1.200000e+07,2.000000e+06,1.00,0.000000,1,1,1,1.000,0.000000,1,1,0,0.000000,0.000000,0,0,0,0,0,0,40,0,2.000000e+06,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,0,9.000000,6.000000,0.0,40,0,0,0,0,0,0,2,12,0,0,245,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
7,49486,1,2,0,12,0,6,6,6.000000,0.000000,0,0,0.0,0.000000,1.200000e+07,2.000000e+06,1.00,0.000000,1,1,1,1.000,0.000000,1,1,0,0.000000,0.000000,0,0,0,0,0,0,40,0,2.000000e+06,0.000000,6,6,6.000000,0.000000,0.000000,0,0,0,0,1,1,0,0,0,9.000000,6.000000,0.0,40,0,0,0,0,0,0,2,12,0,0,245,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
8,88,609,7,4,484,414,233,0,69.142857,111.967895,207,0,103.5,119.511506,1.474548e+06,1.806240e+04,60.90,115.194956,381,2,609,101.5

In [8]:
# check if all the columns in dataframe contains numbers
data.apply(lambda s: pd.to_numeric(s, errors='coerce').notnull().all())

 Destination Port                True
 Flow Duration                   True
 Total Fwd Packets               True
 Total Backward Packets          True
Total Length of Fwd Packets      True
 Total Length of Bwd Packets     True
 Fwd Packet Length Max           True
 Fwd Packet Length Min           True
 Fwd Packet Length Mean          True
 Fwd Packet Length Std           True
Bwd Packet Length Max            True
 Bwd Packet Length Min           True
 Bwd Packet Length Mean          True
 Bwd Packet Length Std           True
Flow Bytes/s                     True
 Flow Packets/s                  True
 Flow IAT Mean                   True
 Flow IAT Std                    True
 Flow IAT Max                    True
 Flow IAT Min                    True
Fwd IAT Total                    True
 Fwd IAT Mean                    True
 Fwd IAT Std                     True
 Fwd IAT Max                     True
 Fwd IAT Min                     True
Bwd IAT Total                    True
 Bwd IAT Mea

In [9]:
data[' Label'].unique()

array(['BENIGN', 'FTPPatator', 'SSHPatator', 'DoS slowloris',
       'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye', 'Heartbleed',
       'Web Attack  Brute Force', 'Web Attack  XSS',
       'Web Attack  Sql Injection', 'Infiltration', 'Bot', 'PortScan',
       'DDoS'], dtype=object)

In [10]:
data.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,3000000.0,5.000000e+05,4.0,0.0,4,4,4,4.0,0.0,4,4,0,0.0,0.0,0,0,0,0,0,0,40,0,5.000000e+05,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4000000.0,6.666667e+05,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,6.666667e+05,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,245,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [11]:
path = '/content/drive/MyDrive/DATASET/GeneratedLabelledFlows/TrafficLabelling /'
import pandas as pd

In [12]:
data1_Mon= pd.read_csv(path +"Monday-WorkingHours.pcap_ISCX.csv")
data2_Tue = pd.read_csv(path+"Tuesday-WorkingHours.pcap_ISCX.csv")
data3_Tue= pd.read_csv(path+"Wednesday-workingHours.pcap_ISCX.csv")
data4_Wed= pd.read_csv(path+"Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv", encoding="ISO-8859-1")
data5_Thurs= pd.read_csv(path+"Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
data6_Fri1= pd.read_csv(path+"Friday-WorkingHours-Morning.pcap_ISCX.csv")
data7_Fri2= pd.read_csv(path+"Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
data8_Fri3= pd.read_csv(path+"Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,3,6,84) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
data = pd.concat([data1_Mon, data2_Tue, data3_Tue, data4_Wed, data5_Thurs, data6_Fri1, data7_Fri2, data8_Fri3], ignore_index=True)

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train, test_Mon = train_test_split(data1_Mon, test_size=0.01, random_state=42, shuffle=True)
test_Mon.to_csv('test_Mon.csv')


In [16]:
data2_Tue = pd.concat([data2_Tue, data3_Tue], ignore_index=True)

In [17]:
train, test_Tue = train_test_split(data2_Tue, test_size=0.01, random_state=42, shuffle=True)
test_Tue.to_csv('test_Tue.csv')


In [18]:
train, test_Wed = train_test_split(data4_Wed, test_size=0.01, random_state=42, shuffle=True)
test_Wed.to_csv('test_Wed.csv')


In [19]:
train, test_Thu = train_test_split(data5_Thurs, test_size=0.01, random_state=42, shuffle=True)
test_Thu.to_csv('test_Thu.csv')


In [20]:
data9_Fri = pd.concat([data6_Fri1,data7_Fri2,data8_Fri3], ignore_index=True)

In [21]:
train, test_Fri = train_test_split(data9_Fri, test_size=0.01, random_state=42, shuffle=True)
test_Fri.to_csv('test_Fri.csv')


In [22]:
data.shape

(3119345, 85)

In [24]:
train, test_all = train_test_split(data, test_size=0.01, random_state=42, shuffle=True)
test_all.to_csv('test_all.csv')
